In [1]:
import numpy as np
import pandas as pd

In [2]:
Movie_data = pd.read_csv('IMDBdata_MainData.csv', low_memory=False)
Movie_data.head(2)

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,tomatoURL
0,Code Name: K.O.Z.,2015,NOT RATED,13 Feb 2015,114 min,"Crime, Mystery",Celal Çimen,NaN,"Cem Kurtoglu, Hakan Ural, Hazim Körmükçü, Tolg...",A look at the 17-25 December 2013 corruption s...,...,1.6,"24,600",tt4458206,movie,NaN,NaN,NaN,NaN,True,NaN
1,Saving Christmas,2014,PG,14 Nov 2014,80 min,"Comedy, Family",Darren Doane,"Darren Doane, Cheston Hervey","Kirk Cameron, Darren Doane, Bridgette Cameron,...",Kirk is enjoying the annual Christmas party ex...,...,1.6,"12,686",tt4009460,movie,03 Nov 2015,"$2,778,297",IPD/Samuel Goldwyn Films,http://www.savingchristmas.com/,True,http://www.rottentomatoes.com/m/kirk_camerons_...


In [3]:
Movie_data.describe()

,Year,Metascore,imdbRating
count,5273.000000,3876.000000,5273.000000
mean,2001.766547,54.908927,6.439522
std,13.929189,18.502623,1.253977
min,1916.000000,1.000000,1.200000
25%,1998.000000,41.000000,5.800000
50%,2005.000000,55.000000,6.600000
75%,2011.000000,69.000000,7.300000
max,2017.000000,100.000000,9.300000


In [4]:
Movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5273 entries, 0 to 5272
Data columns (total 27 columns):
Title             5273 non-null object
Year              5273 non-null int64
Rated             4815 non-null object
Released          5235 non-null object
Runtime           5257 non-null object
Genre             5271 non-null object
Director          5272 non-null object
Writer            5221 non-null object
Actors            5266 non-null object
Plot              5262 non-null object
Language          5263 non-null object
Country           5271 non-null object
Awards            4462 non-null object
Poster            5260 non-null object
Ratings.Source    5273 non-null object
Ratings.Value     5273 non-null object
Metascore         3876 non-null float64
imdbRating        5273 non-null float64
imdbVotes         5273 non-null object
imdbID            5273 non-null object
Type              5273 non-null object
DVD               4882 non-null object
BoxOffice         2634 non-null ob

In [5]:
#Print plot overviews of the first 5 movies.
Movie_data['Plot'].head()

0    A look at the 17-25 December 2013 corruption s...
1    Kirk is enjoying the annual Christmas party ex...
2    A group of smart-talking toddlers find themsel...
3    Evil assassins want to kill Daniel Kublbock, t...
4    A family gets lost on the road and stumbles up...
Name: Plot, dtype: object

In [6]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

In [7]:
#Replace NaN with an empty string
Movie_data['Plot'] = Movie_data['Plot'].fillna('')

In [8]:
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(Movie_data['Plot'])

In [9]:
#Output the shape of tfidf_matrix
tfidf_matrix.shape

(5273, 14266)

You see that over 14,000 different words were used to describe the 5,273 movies in your dataset.

With this matrix in hand, you can now compute a similarity score. There are several candidates for this; such as:

1) the euclidean, 
2) the Pearson 
3) and the cosine similarity scores. 

Again, there is no right answer to which score is the best. Different scores work well in different scenarios and it is often a good idea to experiment with different metrics.

In [10]:
print(tfidf_matrix)

  (0, 5536)	0.21467240224923734
  (0, 4464)	0.35515930543460705
  (0, 13667)	0.35515930543460705
  (0, 13214)	0.31850113040005346
  (0, 11175)	0.28695720343661324
  (0, 2975)	0.24550612457873058
  (0, 178)	0.35515930543460705
  (0, 3382)	0.3274284665362348
  (0, 203)	0.29498546479806004
  (0, 32)	0.29498546479806004
  (0, 7596)	0.22405010498479594
  (1, 6241)	0.3076284406470934
  (1, 1187)	0.3076284406470934
  (1, 2129)	0.19678482256447066
  (1, 1176)	0.22783696047046093
  (1, 7356)	0.1960847381570171
  (1, 1838)	0.17172030041490927
  (1, 2384)	0.24553860057768512
  (1, 5970)	0.15018964863063172
  (1, 8681)	0.23556955385231046
  (1, 10336)	0.23346872886017436
  (1, 11661)	0.18297078052272026
  (1, 12841)	0.23780600719583558
  (1, 4706)	0.3076284406470934
  (1, 9288)	0.192781954539757
  :	:
  (5270, 7387)	0.17076110475910183
  (5270, 2081)	0.17291246872970095
  (5270, 14200)	0.1434618800952129
  (5270, 1931)	0.17076110475910183
  (5270, 11870)	0.1599000627903116
  (5270, 4951)	0.1301049

You will be using the cosine similarity to calculate a numeric quantity that denotes the similarity between two movies. You use the cosine similarity score since it is independent of magnitude and is relatively easy and fast to calculate (especially when used in conjunction with TF-IDF scores, which will be explained later). Mathematically, it is defined as follows:

cosine(x,y)= x.y⊺ / ||x||.||y||

Since you have used the TF-IDF vectorizer, calculating the dot product will directly give you the cosine similarity score. Therefore, you will use sklearn's linear_kernel() instead of cosine_similarities() since it is faster.


In [11]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [12]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(Movie_data.index, index=Movie_data['Title']).drop_duplicates()

You are now in a good position to define your recommendation function. These are the following steps you'll follow:

1) Get the index of the movie given its title.

2) Get the list of cosine similarity scores for that particular movie with all movies. Convert it into a list of tuples where the first element is its position and the second is the similarity score.

3) Sort the aforementioned list of tuples based on the similarity scores; that is, the second element.

4) Get the top 10 elements of this list. Ignore the first element as it refers to self (the movie most similar to a particular movie is the movie itself).

5) Return the titles corresponding to the indices of the top elements.

In [13]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return Movie_data['Title'].iloc[movie_indices]

In [16]:
get_recommendations('The Godfather')

2394                    Blood Ties
4891                 American Hero
4128                          Made
923                 Little Fockers
1710    Curse of the Golden Flower
1802                         Spawn
2390                     Self/less
2641            Death at a Funeral
5020                  Diamond Ruff
4260                The Jerky Boys
Name: Title, dtype: object

In [15]:
get_recommendations('The Dark Knight Rises')

1960                             Batman
103                     The Dark Knight
4674                                Yes
1218                          The Siege
2239                            Godsend
2458                          Elizabeth
3749    The Wind That Shakes the Barley
1064                     Batman Returns
2122              George and the Dragon
3249                            Flipped
Name: Title, dtype: object

Above we have successfully built a recommender system which reccomends a movie based on resembles to plot.

Now our aim is to create a recommender system of simmilar movie director. 